# Reverifying Results from Quintanilla et al. (2000)

Quintanilla, Torquato, and Ziff (2000) developed a highly efficient Monte Carlo algorithm for measuring the **percolation threshold of fully penetrable (overlapping) disks** in two dimensions. Their high-precision threshold estimates have since become a standard reference in continuum percolation theory.

While their work concerns overlapping disks rather than Random Geometric Graphs directly, the two settings are closely related. As noted by Mathew Penrose (2003), this result translates to a critical mean degree of approximately **4.52** for percolation in RGGs. Using our RGG library, we generate the giant component transition curves across mean degrees $k \in [4, 6]$ for system sizes $n \in [1024, 4096, 65536]$, averaging over **50 independent instances** per $(n, k)$ pair, as a check of our implementation against this well-established baseline. We also want to understand how fast $\frac{LCC}{n}$ grows with respect to the mean degree.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
from RGG_Library import RGGBuilder

# ---------------------------
# PARAMETERS
# ---------------------------
space_        = "torus"
perturb_bool  = False
n_instances   = 50
k_values      = np.arange(4.0, 6.01, 0.01)
n_values      = [1024, 4096, 65536]

print("n values:", n_values)
print(f"k range: {k_values[0]:.2f} → {k_values[-1]:.2f}  ({len(k_values)} steps)")
print(f"Instances per (n,k): {n_instances}")
print(f"Total graphs to build: {len(n_values) * len(k_values) * n_instances:,}\n")

# ---------------------------
# HELPER: return giant component size as fraction of n
# ---------------------------
def giant_component_fraction(n, k, seed):
    try:
        builder = RGGBuilder(
            n=n, k=k, connectivity_regime="sc",
            space=space_, order=4, perturb=perturb_bool, seed=seed
        )
        G = builder.build()
        if G.number_of_nodes() == 0:
            return 0.0
        largest = max(nx.connected_components(G), key=len)
        return len(largest) / n
    except Exception:
        return np.nan

# ---------------------------
# SWEEP
# ---------------------------
# gc_mean[i_n, i_k] = mean giant component fraction over n_instances
gc_mean = np.full((len(n_values), len(k_values)), np.nan)

for i, n in enumerate(n_values):
    print(f"[n={n}]  sweeping k ...", flush=True)
    for j, k in enumerate(k_values):
        fractions = [giant_component_fraction(n, k, seed=s) for s in range(n_instances)]
        gc_mean[i, j] = np.nanmean(fractions)

    print(f"  done. GC range: [{np.nanmin(gc_mean[i]):.3f}, {np.nanmax(gc_mean[i]):.3f}]")

# ---------------------------
# BUILD DATAFRAME
# ---------------------------
records = []
for i, n in enumerate(n_values):
    for j, k in enumerate(k_values):
        records.append({
            "n":            n,
            "k":            round(k, 2),
            "ln(n)":        np.log(n),
            "gc_fraction":  gc_mean[i, j],
            "above_threshold": k >= np.log(n),
        })

df = pd.DataFrame(records)

print("\nResults DataFrame (first 10 rows):")
print(df.head(10).to_string(index=False))

df.to_csv("LCC_Results.csv", index=False)
print("\nDataFrame saved to LCC_Results.csv")

# ---------------------------
# PLOT
# ---------------------------
fig, ax = plt.subplots(figsize=(11, 6))

cmap   = cm.plasma
log_ns = np.log2(np.array(n_values))
norm   = plt.Normalize(vmin=log_ns.min(), vmax=log_ns.max())

for i, n in enumerate(n_values):
    color = cmap(norm(log_ns[i]))
    ax.plot(
        k_values, gc_mean[i],
        color=color, linewidth=2,
        label=f"n = {n:,}"
    )

# Reference line at GC = 1.0 (fully connected)
ax.axhline(1.0, color='k', linewidth=1, linestyle='--', alpha=0.4, label='GC = n')

# Colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label(r'$\log_2(n)$', fontsize=12)
cbar.set_ticks(log_ns)
cbar.set_ticklabels([f"{n:,}" for n in n_values])

ax.set_xlabel("Mean Degree  $k$", fontsize=13)
ax.set_ylabel("Largest Component Size  (fraction of $n$)", fontsize=13)
ax.set_title(
    f"Giant Component Transition vs Mean Degree $k$\n"
    f"({n_instances} instances per point, torus RGG)",
    fontsize=12
)
ax.set_xlim(k_values[0], k_values[-1])
ax.set_ylim(-0.02, 1.05)
ax.legend(fontsize=8, loc='upper left', ncol=2)
ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig("LCC_Plot.png", dpi=150)
plt.show()

print("\nDone. LCC_Plot.png")